<a href="https://colab.research.google.com/github/pradyot-09/DL-Reproducibility-Project/blob/master/distillation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Importing libraries

In [0]:
# Import pytorch basic functions/classes
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim

# Import torchvision functions/classes for MNIST import and data loaders
import torchvision
import torchvision.transforms as transforms

# Set device on which code is run
device = 'cuda'

Defining support functions

In [0]:
# Define support function used to convert label to one-hot encoded tensor
def convert_labels(labels):
    target = torch.zeros([len(labels), 10], dtype=torch.float32)
    for i, l in enumerate(labels):
      target[i][l] = 1.0
    return target

Define our network model (the hidden layers size is specified through the constructor)

In [0]:
# Define MLP model and its layers
class Model(nn.Module):

    def __init__(self, hidden_size=1200, dropout=0.0, hidden_dropout=0.0):
        super(Model, self).__init__()
        self.dropout = nn.Dropout(dropout)
        self.hidden1 = nn.Linear(784, hidden_size, bias=True)
        self.hidden1_dropout = nn.Dropout(hidden_dropout)
        self.hidden2 = nn.Linear(hidden_size, hidden_size, bias=True)
        self.hidden2_dropout = nn.Dropout(hidden_dropout)
        self.hidden3 = nn.Linear(hidden_size, 10, bias=True)

    def forward(self, x):
        x = self.dropout(x)
        x = F.relu(self.hidden1(x))
        x = self.hidden1_dropout(x)
        x = F.relu(self.hidden2(x))
        x = self.hidden2_dropout(x)
        x = self.hidden3(x)
        return x#, F.softmax(x)

Downloading MNIST dataset

In [0]:
# Define transform from PIL image to tensor and normalize to 1x768 pixels
transform = transforms.Compose([
  transforms.ToTensor(),
  transforms.Normalize((0.5,), (0.5,))
])

# Set batch size for data loaders
batch_size = 128

# (Down)load training set
trainset = torchvision.datasets.MNIST(root='./data', train=True, download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size, shuffle=True, num_workers=2)

# (Down)load test set
testset = torchvision.datasets.MNIST(root='./data', train=False, download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size, shuffle=False, num_workers=2)

Extracting ./data/MNIST/raw/train-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/train-labels-idx1-ubyte.gz to ./data/MNIST/raw



Extracting ./data/MNIST/raw/t10k-images-idx3-ubyte.gz to ./data/MNIST/raw


Extracting ./data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ./data/MNIST/raw
Processing...
Done!


Training the Deep Teacher Neural Network

In [0]:
# Setup model and move it to the GPU
net = Model(dropout=0.2, hidden_dropout=0.5)
net.to(device)

# Set up loss function and optimizer
criterion = torch.nn.MSELoss()#CrossEntropyLoss()
#optimizer = optim.SGD(net.parameters(), lr=0.1, momentum=0.9)
optimizer = torch.optim.Adam(net.parameters(), lr=0.0001, weight_decay=0.00001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100): #200 for SGD reaches 0.002 loss

    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        

        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, target)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
PATH = './mnist_dropout_100_epoch.pth'
torch.save(net.state_dict(), PATH)

print('Saved Model')

[1] loss: 0.023
[2] loss: 0.014
[3] loss: 0.011
[4] loss: 0.010
[5] loss: 0.009
[6] loss: 0.008
[7] loss: 0.007
[8] loss: 0.007
[9] loss: 0.007
[10] loss: 0.006
[11] loss: 0.006
[12] loss: 0.006
[13] loss: 0.005
[14] loss: 0.005
[15] loss: 0.005
[16] loss: 0.005
[17] loss: 0.005
[18] loss: 0.005
[19] loss: 0.004
[20] loss: 0.004
[21] loss: 0.004
[22] loss: 0.004
[23] loss: 0.004
[24] loss: 0.004
[25] loss: 0.004
[26] loss: 0.004
[27] loss: 0.004
[28] loss: 0.004
[29] loss: 0.004
[30] loss: 0.004
[31] loss: 0.003
[32] loss: 0.003
[33] loss: 0.003
[34] loss: 0.003
[35] loss: 0.003
[36] loss: 0.003
[37] loss: 0.003
[38] loss: 0.003
[39] loss: 0.003
[40] loss: 0.003
[41] loss: 0.003
[42] loss: 0.003
[43] loss: 0.003
[44] loss: 0.003
[45] loss: 0.003
[46] loss: 0.003
[47] loss: 0.003
[48] loss: 0.003
[49] loss: 0.003
[50] loss: 0.003
[51] loss: 0.003
[52] loss: 0.003
[53] loss: 0.003
[54] loss: 0.003
[55] loss: 0.003
[56] loss: 0.003
[57] loss: 0.003
[58] loss: 0.003
[59] loss: 0.003
[60] l

Run Deep Teacher model on test set

In [0]:
# Instantiate model and load saved network parameters
net = Model().to(device)
net.load_state_dict(torch.load(PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 98 % (145 wrong out of 10000)


Train student model to mimic the teacher

In [0]:
# Custom student loss: linear combination of 2 MSE losses
#     The first one between student output and hard labels
#     The second one between student output and soft labels from teacher

def student_loss(outputA, targetA, outputB, targetB, weight):

    loss = torch.mean((weight*(outputA - targetA)**2) + (1-weight)*(outputB-targetB)**2)

    return loss

# Setup student model and move it to the GPU
student_net = Model(hidden_size = 800)
student_net.to(device)

# Set up loss function and optimizer

#optimizer = optim.SGD(student_net.parameters(), lr=0.001, momentum=0.9)
optimizer = torch.optim.Adam(student_net.parameters(), lr=0.0001)

# Run over 100 epochs (1 epoch = visited all items in dataset)
for epoch in range(100):
    running_loss = 0.0
    total = 0
    for i, data in enumerate(trainloader, 0):
        # get the inputs; data is a list of [inputs, labels]
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        
        # zero the parameter gradients
        optimizer.zero_grad()

        # Set temperature and the weights for losses linear combination
        w = 0.3
        T = 20

        # Compute soft labels using deep teacher model previously trained
        outputs_teacher = net(inputs)
        soft_labels = F.softmax(outputs_teacher/T, dim = 1)

        # Student forward + backward + optimize
        outputs_stud = student_net(inputs)
        loss = student_loss(outputs_stud, target, outputs_stud, soft_labels, w)
        loss.backward()
        optimizer.step()

        total += len(data)

        # print statistics
        running_loss += loss.item()
    # print every epoch
    print('[%d] loss: %.3f' % (epoch + 1, running_loss / total))

print('Finished Training')

# Save model after having finished training
STUD_PATH = './mnist_student_100_epoch.pth'
torch.save(student_net.state_dict(), STUD_PATH)

print('Saved Model')

[1] loss: 0.010
[2] loss: 0.010
[3] loss: 0.010
[4] loss: 0.010
[5] loss: 0.010
[6] loss: 0.010
[7] loss: 0.010
[8] loss: 0.010
[9] loss: 0.010
[10] loss: 0.010
[11] loss: 0.009
[12] loss: 0.009
[13] loss: 0.009
[14] loss: 0.009
[15] loss: 0.009
[16] loss: 0.009
[17] loss: 0.009
[18] loss: 0.009
[19] loss: 0.009
[20] loss: 0.009
[21] loss: 0.009
[22] loss: 0.009
[23] loss: 0.009
[24] loss: 0.009
[25] loss: 0.009
[26] loss: 0.009
[27] loss: 0.009
[28] loss: 0.009
[29] loss: 0.009
[30] loss: 0.009
[31] loss: 0.009
[32] loss: 0.009
[33] loss: 0.009
[34] loss: 0.009
[35] loss: 0.009
[36] loss: 0.009
[37] loss: 0.009
[38] loss: 0.009
[39] loss: 0.009
[40] loss: 0.009
[41] loss: 0.009
[42] loss: 0.009
[43] loss: 0.009
[44] loss: 0.009
[45] loss: 0.009
[46] loss: 0.009
[47] loss: 0.009
[48] loss: 0.009
[49] loss: 0.009
[50] loss: 0.009
[51] loss: 0.009
[52] loss: 0.009
[53] loss: 0.009
[54] loss: 0.009
[55] loss: 0.009
[56] loss: 0.009
[57] loss: 0.009
[58] loss: 0.009
[59] loss: 0.009
[60] l

Running student model on test set

In [0]:
stud_net = Model(hidden_size = 800).to(device)
stud_net.load_state_dict(torch.load(STUD_PATH))

# Run model on test set and determine accuracy
correct = 0
total = 0
with torch.no_grad():
    for data in testloader:
        inputs, labels = data
        inputs = torch.flatten(inputs, start_dim=1).to(device)
        target = convert_labels(labels).to(device)
        outputs = stud_net(inputs)
        _, predicted = torch.max(outputs.data, 1)
        _, target = torch.max(target.data, 1)
        total += target.size(0)
        correct += (predicted == target).sum().item()

# Output model accuracy to user
print('Accuracy of the network on the 10000 test images: %d %% (%d wrong out of %d)' % (
    100 * correct / total, total - correct, total))

Accuracy of the network on the 10000 test images: 98 % (142 wrong out of 10000)
